In [4]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Dataframes
command: disjoint speaker split

In [6]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/wav/'
data_df = pd.read_pickle('/home/muncok/DL/dataset/SV_sets/dataframes/Reddots_Dataframe.pkl')

In [7]:
data_df.head(5)

,spk,sent,file,set
20150401220415579_m0045_40,m0045,40,20150401220415579_m0045_40.wav,
20150401220405767_m0045_32,m0045,32,20150401220405767_m0045_32.wav,
20150401220349980_m0045_36,m0045,36,20150401220349980_m0045_36.wav,
20150401220418653_m0045_17449,m0045,17449,20150401220418653_m0045_17449.wav,
20150401220403099_m0045_566,m0045,566,20150401220403099_m0045_566.wav,


In [8]:
import honk_sv.train as hk
from honk_sv import model as mod

### Dataset split

In [9]:
#TODO: correct splits
all_spks = data_df.spk.unique()
all_sents = data_df.sent.unique()
uttrs_counts = data_df.spk.value_counts()
sv_spks = list(uttrs_counts.index[-8:])
si_spks = list(uttrs_counts.index[:-8])

# SI Train

In [10]:
import random 

si_df = data_df[data_df.spk.isin(si_spks)]

# random sampling
si_random_train = si_df.sample(frac=0.8)
si_random_test = si_df.drop(index=si_random_train.index)
si_random_val = si_random_test.sample(frac=0.5)
si_random_test = si_random_test.drop(index=si_random_val.index) 
print("[random] train:{}, val:{}, test:{}".format(len(si_random_train), len(si_random_val), len(si_random_test)))

[random] train:12090, val:1512, test:1511


choose 2300 as the total number of utterances 

In [21]:
import csv
import random

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_random_train, 'val':si_random_val, 'test':si_random_test}
manifest_dir = "manifests/reddots/"

for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = si_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

manifests/reddots/si_reddots_train_manifest.csv was written
manifests/reddots/si_reddots_val_manifest.csv was written
manifests/reddots/si_reddots_test_manifest.csv was written


# SV Train

In [12]:
import random
random.shuffle(sv_spks)
enroll_spks = sv_spks[:5]
test_spks = sv_spks[5:]

In [13]:
enroll_uttrs = pd.DataFrame()
dev_uttrs = pd.DataFrame()
enroll_pts = 0.3

sv_df = data_df[data_df.spk.isin(sv_spks)]
# splits enroll and dev
for spk in enroll_spks:
    spk_df = sv_df[sv_df.spk == spk]
    assert(len(spk_df) != 0)
    enls = spk_df.sample(frac=enroll_pts)
    devs = spk_df.drop(index=enls.index)
    enroll_uttrs = pd.concat([enls, enroll_uttrs])
    dev_uttrs = pd.concat([devs, dev_uttrs])

In [14]:
import csv
import random

manifest_dir = "./manifests/reddots/enroll/"
# delete existing files
for file in os.listdir(manifest_dir):
    file_path = os.path.join(manifest_dir, file)
    if os.path.isfile(file_path):
        os.unlink(file_path)
        

for spk in enroll_spks:
    samples = []
    save_path = os.path.join(manifest_dir,'enroll_{}_{}_manifest.csv'.format("reddots", spk))
    with open(save_path, 'w') as f:
        for index, row in enroll_uttrs[enroll_uttrs.spk == spk].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = enroll_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))    

./manifests/reddots/enroll/enroll_reddots_m0056_manifest.csv was written
./manifests/reddots/enroll/enroll_reddots_m0063_manifest.csv was written
./manifests/reddots/enroll/enroll_reddots_m0045_manifest.csv was written
./manifests/reddots/enroll/enroll_reddots_f0015_manifest.csv was written
./manifests/reddots/enroll/enroll_reddots_m0025_manifest.csv was written


In [23]:
model = "res15"
dataset = "reddots"

global_config = dict(model=model, dataset=dataset,
                     no_cuda=False,  gpu_no=0,
                     n_epochs=30, batch_size=64,
                     lr=[0.001], schedule=[np.inf], dev_every=1, seed=0, use_nesterov=False,
                     cache_size=32768, momentum=0.9, weight_decay=0.00001,
                     num_workers=32, print_step=1,
                     splice_length = 20, n_labels =62
                     )

builder = hk.ConfigBuilder(
                mod.find_config(model),
                mod.SpeechDataset.default_config(dataset),
                global_config)
parser = builder.build_argparse()
si_config = builder.config_from_argparse(parser)
si_config['model_class'] = mod.find_model(model)

In [24]:
si_model = si_config['model_class'](si_config)
si_model.load("models/si_reddots_frames.pt")

In [25]:
for tag in ['train', 'val', 'test']:
    si_config['{}_manifest'.format(tag)] =  os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))

In [35]:
# enrolling speakers
# generating speaker models
si_config['batch_size'] = 1
manifest_dir = "./manifests/reddots/enroll/"
uni_spk_models = dict()
for spk in enroll_spks:
    manifest_path = os.path.join(manifest_dir, 'enroll_{}_{}_manifest.csv'.format("reddots", spk))
    si_config['test_manifest'] =  manifest_path
    uni_spk_models[spk] = hk.enroll_frames(si_config, model=si_model)

795
624
566
665
732


In [36]:
from scipy.spatial.distance import cosine
# fourth
for spk in enroll_spks:
    target = uni_spk_models[spk]
    for ref in enroll_spks:
        if ref == spk: continue
        score = 1-cosine(uni_spk_models[ref], target)
        print("{}, {}: {:.2f}".format(spk, ref, score))

m0056, m0063: -0.40
m0056, m0045: -0.17
m0056, f0015: 0.15
m0056, m0025: 0.19
m0063, m0056: -0.40
m0063, m0045: 0.33
m0063, f0015: 0.13
m0063, m0025: 0.11
m0045, m0056: -0.17
m0045, m0063: 0.33
m0045, f0015: -0.15
m0045, m0025: -0.21
f0015, m0056: 0.15
f0015, m0063: 0.13
f0015, m0045: -0.15
f0015, m0025: 0.84
m0025, m0056: 0.19
m0025, m0063: 0.11
m0025, m0045: -0.21
m0025, f0015: 0.84


# SI Train

In [98]:
correct = 0
for i, row in dev_uttrs.iterrows():
    audio_path = os.path.join(base_dir, row.sent, row.file)
    audio_input = torch.zeros((512, 100))
    audio_input.narrow(1,0, 98).copy_(SpectrogramParser.parse_audio_ext(parser, audio_path))
    audio_input = Variable(audio_input.cuda())
    output = model(audio_input, 'fc7')
    emb = output[0].data.cpu().numpy()
    label = sv_spks.index(row.spk)
    embeddings[label] += emb
    pred_spk, score = spk_verify(speaker_model, emb)
    if pred_spk == label:
        correct += 1
print(correct / len(dev_uttrs))

NameError: name 'base_dir' is not defined